In [2]:
#!/usr/bin/python
################################################################################
#Author: Antsa Raharimanantsoa
#Description: Classification using clustering algorithm
#Creation_date: March 2017
################################################################################

import nltk
from mongoengine import *
from document import *
from lib import *
from algo_clustering import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import cluster
from operator import itemgetter
from itertools import groupby
import logging
from datetime import datetime
import time

log_name = datetime.now().strftime("%Y%m%d_%H%M")
logging.basicConfig(filename='log/clusters/' + log_name + '.log',
                    level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    datefmt='%d-%m-%Y %I:%M:%S %p')

stopwords = nltk.corpus.stopwords.words('french')
stopwords += nltk.corpus.stopwords.words('english')

"""Retrieve all contents for the clustering"""
content = get_content_article()
logging.info("Retrieve all articles for the classification")

"""tf-idf representation"""
tfidf_vectorizer = TfidfVectorizer(#max_df=0.8,
                                max_features=200000,
                                 #min_df=0.2,
                                 stop_words=stopwords,
                                use_idf=True,ngram_range=(1,3)
                                ,tokenizer=tokenize_only)
tfidf_matrix = tfidf_vectorizer.fit_transform(content.values())

print('---- TF-IDF done ----')
logging.info("TF-IDF done, clustering ongoing ...")

"""Test of Kmeans (number of clusters must be detected in advance)"""
km_clusters = kmeans(tfidf_matrix, 5)
gp_tokens = []
gp_clusters = [(content.keys()[idx],
                vlue) for (idx,vlue) in enumerate(km_clusters)]
sorted_cl = sorted(gp_clusters, key=itemgetter(1))
gather_cl = groupby(sorted_cl, key=itemgetter(1))
gp_kmeans = [{'kmeans_cl':k,
            'obj_id':[elms1 for (elms1,elms2) in v]} for k, v in gather_cl]

#print(len(gp_kmeans))

stopwords = nltk.corpus.stopwords.words('french')
stopwords += nltk.corpus.stopwords.words('english')

connect(DATABASE_NAME)
texts=[]
for elms in gp_kmeans:
    tab = []
    for val in elms['obj_id']:
        #tab.append(dict((val,el.tokens) for el in NewArticle.objects(id__exact=val)))
        tokens = [el.tokens for el in NewArticle.objects(id__exact=val)]
        tab.append((val,''.join(tokens)))
    new_content = dict(tab)        
    tfidf_2 = TfidfVectorizer(max_df=0.8,
                                max_features=200000,
                                 min_df=0.2,
                                 stop_words=stopwords,
                                use_idf=True,ngram_range=(1,3)
                                ,tokenizer=tokenize_only)
    tfidf_matrix2 = tfidf_vectorizer.fit_transform(new_content.values())
      
    dist = 1 - cosine_similarity(tfidf_matrix2)
    #bandwidth = cluster.estimate_bandwidth(dist, quantile=0.25)
    #ms = cluster.MeanShift(bandwidth=bandwidth)
    #ms.fit(dist)
    #clusters = ms.labels_
    #cluster_centers = ms.cluster_centers_
    
    ti = []
    for valeur in new_content.keys():
        ti += [atcl.title for atcl in NewArticle.objects(id__exact=valeur)]
    hierarchical(dist,ti)

    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    

/home/antsa/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


---- TF-IDF done ----


TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]